
"""
CHECK

- The Missing Value Treatement for the rtgp (runtime in group) variables - espcially for races where NONE of the dogs have a runtime

More Variables :

Total Races from current race
- Races in the last 7/14/28/60/90/180/365/730 days 
- Races won in the 7/14/28/60/90/180/365/730 days
- Races placed in the 7/14/28/60/90/180/365/730 days


"""


# Import the code 


In [1]:

#fileloc  = '/mnt/batch/tasks/shared/LS_root/mounts/clusters/compute-one/code/Users/karan.bhatti'
fileloc  = '/mnt/batch/tasks/shared/LS_root/mounts/clusters/compute-one-8c-32gb/code/Users/karan.bhatti'

import sys
print(sys.path)
print()
#sys.path.append(f'{fileloc}')
print()
#print(sys.path)


['/mnt/batch/tasks/shared/LS_root/mounts/clusters/compute-one-8c-32gb/code/Users/karan.bhatti', '/anaconda/envs/azureml_py38/lib/python38.zip', '/anaconda/envs/azureml_py38/lib/python3.8', '/anaconda/envs/azureml_py38/lib/python3.8/lib-dynload', '', '/anaconda/envs/azureml_py38/lib/python3.8/site-packages', '/anaconda/envs/azureml_py38/lib/python3.8/site-packages/azureml/_project/vendor', '/anaconda/envs/azureml_py38/lib/python3.8/site-packages/azureml/automl/core/_vendor']




In [2]:

class Data(object):
    pass

df = Data
summ = Data


In [3]:
import helpdesk as hd

In [4]:

import pandas as pd, numpy as np
from datetime import datetime


In [5]:
DEBUG = True

In [6]:

pd.set_option('display.max_rows', 50);
pd.set_option('display.max_columns', 50);
pd.set_option('display.width', 200);


In [7]:

#fname_race = 'race_details_2022Q1_2022Q4.csv'
#fname_dogs = 'dog_results_2022Q1_2022Q2.csv'

#race_details = pd.read_csv(f'{fileloc}/{fname_race}')
#dogs_details = pd.read_csv(f'{fileloc}/{fname_dogs}')

#print(race_details.shape)
#print(dogs_details.shape)


## IMPORTING THE RAW DATA AND MERGE

In [8]:

# Importing Dog Race Results

#res_raw = pd.read_csv('C:\\Users\\karan\\Documents\\Data\\racing\\FastTrack\\Past\\dog_results_20211201_20211231.csv')
#df.res_raw = pd.read_csv('C:\\Users\\karan\\Documents\\Data\\racing\\FastTrack\\Past\\dog_results_20220101_20220531.csv')

print(f'{fileloc}/dog_results_2022Q1_2022Q2.csv')

df.res_raw_2021Q1Q4 = pd.read_csv(f'{fileloc}/dog_results_2021Q1_2021Q4.csv')
print(df.res_raw_2021Q1Q4.columns.values.tolist())
print(df.res_raw_2021Q1Q4.shape)

df.res_raw_2022Q1Q2 = pd.read_csv(f'{fileloc}/dog_results_2022Q1_2022Q2.csv')
print(df.res_raw_2022Q1Q2.columns.values.tolist())
print(df.res_raw_2022Q1Q2.shape)



/mnt/batch/tasks/shared/LS_root/mounts/clusters/compute-one-8c-32gb/code/Users/karan.bhatti/dog_results_2022Q1_2022Q2.csv
['@id', 'Place', 'DogName', 'Box', 'Rug', 'Weight', 'StartPrice', 'Handicap', 'Margin1', 'Margin2', 'PIR', 'Checks', 'Comments', 'SplitMargin', 'RunTime', 'Prizemoney', 'RaceId', 'TrainerId', 'TrainerName']
(435715, 19)
['@id', 'Place', 'DogName', 'Box', 'Rug', 'Weight', 'StartPrice', 'Handicap', 'Margin1', 'Margin2', 'PIR', 'Checks', 'Comments', 'SplitMargin', 'RunTime', 'Prizemoney', 'RaceId', 'TrainerId', 'TrainerName']
(320052, 19)


In [9]:

df.res_raw = df.res_raw_2021Q1Q4.append( df.res_raw_2022Q1Q2 )
print(df.res_raw.columns.values.tolist())
print(df.res_raw.shape)


['@id', 'Place', 'DogName', 'Box', 'Rug', 'Weight', 'StartPrice', 'Handicap', 'Margin1', 'Margin2', 'PIR', 'Checks', 'Comments', 'SplitMargin', 'RunTime', 'Prizemoney', 'RaceId', 'TrainerId', 'TrainerName']
(755767, 19)


In [10]:

df.res_raw.loc[:,"DogName"] = df.res_raw["DogName"].str.replace("'","").str.replace(".","")
if DEBUG: print(df.res_raw.shape) 
# 320 052

#if DEBUG: res_raw[ ~(res_raw.Place.isin( ['D','F','N','R','S','T',''])) ].shape # 66608
if DEBUG: print(df.res_raw[ ~(df.res_raw.Place.isin( ['R','S'])) ].shape) 
# 275 864

df.res_raw2 = df.res_raw[ ~(df.res_raw.Place.isin( ['R','S']))  ]
df.res_ = df.res_raw2[~df.res_raw2.Place.isna()]
if DEBUG: print(df.res_.shape) 
# 273 735

del df.res_raw, df.res_raw2

#res_ = res_raw


(755767, 19)
(649399, 19)
(644761, 19)


In [11]:

# Importing Race Details 

#race_raw = pd.read_csv('C:\\Users\\karan\\Documents\\Data\\racing\\FastTrack\\Past\\race_details_20211201_20211231.csv', parse_dates = True)
#df.race_raw = pd.read_csv('C:\\Users\\karan\\Documents\\Data\\racing\\FastTrack\\Past\\race_details_20220101_20220531.csv', parse_dates = True)

print(f'{fileloc}/race_details_2022Q1_2022Q2.csv')

df.race_raw_2022Q1Q2 = pd.read_csv(f'{fileloc}/race_details_2022Q1_2022Q2.csv', parse_dates = True)
print(df.race_raw_2022Q1Q2.columns.values.tolist())
print(df.race_raw_2022Q1Q2.shape)

df.race_raw_2021Q1Q4 = pd.read_csv(f'{fileloc}/race_details_2021Q1_2021Q4.csv', parse_dates = True)
print(df.race_raw_2021Q1Q4.columns.values.tolist())
print(df.race_raw_2021Q1Q4.shape)


/mnt/batch/tasks/shared/LS_root/mounts/clusters/compute-one-8c-32gb/code/Users/karan.bhatti/race_details_2022Q1_2022Q2.csv
['@id', 'RaceNum', 'RaceName', 'RaceTime', 'Distance', 'RaceGrade', 'Track', 'date']
(38562, 8)
['@id', 'RaceNum', 'RaceName', 'RaceTime', 'Distance', 'RaceGrade', 'Track', 'date']
(51833, 8)


In [12]:


df.race_raw = df.race_raw_2021Q1Q4.append( df.race_raw_2022Q1Q2 )
print(df.race_raw.columns.values.tolist())
print(df.race_raw.shape)




df.race_raw['Event_Dt'] = pd.to_datetime(df.race_raw['date']).dt.date

#dtvalct = race_raw.Event_Dt.value_counts()

df.race_raw.loc[df.race_raw.Track.isin(['Murray Bridge (MBR)','Murray Bridge (MBS)']), "Track"] = 'Murray Bridge'
df.race_raw.loc[df.race_raw.Track.isin(['Richmond (RIS)']), "Track"] = 'Richmond'

df.race_raw.loc[df.race_raw.Track.isin(['Sandown (SAP)']), "Track"] = 'Sandown Park'

df.race_raw.loc[df.race_raw.Track.isin(['Meadows (MEP)']), "Track"] = 'Meadows'
df.race_raw.loc[df.race_raw.Track.isin(['The Meadows']), "Track"] = 'Meadows'


if DEBUG : print(df.race_raw.shape)

# 2021
# 52,103 Races

# 2022 upto May
# 21,002 Races

# 2022 Q1/Q2
# 38,562 Races

# 2022 Q1/Q2 + 2021 Q1/Q2/Q3/Q4
# 81,412 Races

df.race_ = df.race_raw[ ~df.race_raw.Track.str.contains('NZ')]

if DEBUG : print(df.race_.shape)

del df.race_raw

#if DEBUG: print(df.race_.dtypes)


['@id', 'RaceNum', 'RaceName', 'RaceTime', 'Distance', 'RaceGrade', 'Track', 'date']
(90395, 8)
(90395, 9)
(81412, 9)


In [13]:

if DEBUG:
    print(df.res_.columns.values.tolist())
    print(df.race_.columns.values.tolist())
    
    print(df.res_.shape)
    print(df.race_.shape)

df.ft_raw = pd.merge(df.res_, df.race_, left_on = 'RaceId', right_on = '@id', how = 'inner', suffixes=('_DOG', '_RACE'))
if DEBUG: print(df.ft_raw.shape)

# 2021
# 333,285

# 2022 upto May
# 132,558

# 2022 Q1/Q2
# 245,827

# 2022 Q1/Q2 + 2021 Q1/Q2/Q3/Q4
# 245,827

#qc_DogName = df.ft_merg.DogName.value_counts()


['@id', 'Place', 'DogName', 'Box', 'Rug', 'Weight', 'StartPrice', 'Handicap', 'Margin1', 'Margin2', 'PIR', 'Checks', 'Comments', 'SplitMargin', 'RunTime', 'Prizemoney', 'RaceId', 'TrainerId', 'TrainerName']
['@id', 'RaceNum', 'RaceName', 'RaceTime', 'Distance', 'RaceGrade', 'Track', 'date', 'Event_Dt']
(644761, 19)
(81412, 9)
(577317, 28)


In [14]:

#summ.ft = hd.describe_df(df.ft)


In [15]:

print(df.ft_raw.columns.values.tolist())


['@id_DOG', 'Place', 'DogName', 'Box', 'Rug', 'Weight', 'StartPrice', 'Handicap', 'Margin1', 'Margin2', 'PIR', 'Checks', 'Comments', 'SplitMargin', 'RunTime', 'Prizemoney', 'RaceId', 'TrainerId', 'TrainerName', '@id_RACE', 'RaceNum', 'RaceName', 'RaceTime', 'Distance', 'RaceGrade', 'Track', 'date', 'Event_Dt']


### A) Prepare the Base Dataset -- Exclusions (Missing ft sp, place info, Margin1, num_dogs u5)

In [18]:

# Delete races where ft_sp is blank
all_races = df.ft_raw['@id_RACE'].unique().tolist()
print(len(all_races))
print()

print("# Part 1 - Removing Races with missing fast track prices - can't be measured for performance")
# Clean the fast track prices
df.ft_raw['ft_sp'] = df.ft_raw.StartPrice.apply(lambda x : float(str(x).replace("$","").replace("F","")) )

blank_ft_sp_races = df.ft_raw[df.ft_raw['ft_sp'].isna()]['@id_RACE'].unique().tolist()
#print(len(blank_ft_sp_races))
df_base0 = df.ft_raw[ ( ~df.ft_raw['@id_RACE'].isin(blank_ft_sp_races) ) ]
print(f"Shape : {df_base0.shape}, Races : {len(df_base0['@id_RACE'].unique().tolist())}, Race w missing fast track prices : {len(blank_ft_sp_races)}")
print()

print("# Part 2 - Removing Races with Missing Place Information")
place_to_remove = ['F','T','N','P','B','D']
races_excl_pos = df_base0[df_base0['Place'].isin(place_to_remove)]['@id_RACE'].unique().tolist()
#print(len(races_excl_pos))

df_base1 = df_base0[ ( ~df_base0['@id_RACE'].isin(races_excl_pos) ) ]
print(f"Shape : {df_base1.shape}, Races : {len(df_base1['@id_RACE'].unique().tolist())}, Races w missing Place : {len(races_excl_pos)}")
print()

print("# Part 3 - Removing Races with no Missing Margin1 information")
races_excl_margin = df_base1[df_base1['Margin1'].isna()]['@id_RACE'].unique().tolist()
#print(len(races_excl_margin))

df_base2 = df_base1[ ( ~df_base1['@id_RACE'].isin(races_excl_margin) ) ]
print(f"Shape : {df_base2.shape}, Races : {len(df_base2['@id_RACE'].unique().tolist())}, Races w missing Margin : {len(races_excl_margin)}")

del df_base0, df_base1


80909

# Part 1 - Removing Races with missing fast track prices - can't be measured for performance
Shape : (569648, 29), Races : 79518, Race w missing fast track prices : 1391

# Part 2 - Removing Races with Missing Place Information
Shape : (541552, 29), Races : 75740, Races w missing Place : 3778

# Part 3 - Removing Races with no Missing Margin1 information
Shape : (541527, 29), Races : 75721, Races w missing Margin : 19


In [19]:

print("# Part 4 - Removing Races with under 5 dogs")
df_num_dogs = df_base2.groupby("@id_RACE").agg( { '@id_DOG':'nunique' } ).reset_index().rename(columns = {'@id_DOG':'x_num_dogs'})

print(df_num_dogs.shape)
print(df_base2.shape)
print()

df_base3 = pd.merge( df_base2, df_num_dogs, on = ["@id_RACE"], how = 'left')
print(df_base3.columns.values.tolist())
print()

races_u5dogs = df_base3[df_base3.x_num_dogs.isin([1,2,3,4])]['@id_RACE'].unique().tolist()
print(len(races_u5dogs))
print()

print( len(df_base3['@id_RACE'].unique().tolist()))
df_base = df_base3[ ( ~df_base3['@id_RACE'].isin(races_u5dogs) ) ].copy()   
print()

#print(df.ft.shape)
print(f"Shape : {df_base.shape}, Races : {len(df_base['@id_RACE'].unique().tolist())}, Races u5 Dogs : {len(races_u5dogs)}")
print()

del df_base2, df_base3


# Part 4 - Removing Races with under 5 dogs
(75721, 2)
(541527, 29)

['@id_DOG', 'Place', 'DogName', 'Box', 'Rug', 'Weight', 'StartPrice', 'Handicap', 'Margin1', 'Margin2', 'PIR', 'Checks', 'Comments', 'SplitMargin', 'RunTime', 'Prizemoney', 'RaceId', 'TrainerId', 'TrainerName', '@id_RACE', 'RaceNum', 'RaceName', 'RaceTime', 'Distance', 'RaceGrade', 'Track', 'date', 'Event_Dt', 'ft_sp', 'x_num_dogs']

1243

75721

Shape : (536782, 30), Races : 74478, Races u5 Dogs : 1243



In [ ]:

#del dognameslist

# DogNames list
dognameslist = df_base.DogName.unique().tolist()

# Fix the Place Variable
df_base['position'] = df_base.Place.apply(lambda x : x.replace("=","") if x in ['1=','2=','3=','4=','5=','6=','7=','8='] else x)

#print(df_base_sp.position.value_counts())


In [20]:
df_base.sort_values(by = ['@id_DOG', 'DogName', 'Event_Dt'], inplace = True)

In [21]:

df_base_sp = df_base.copy()


### B) Create Variable for Speed Profile

In [22]:

# Sort for Easy QC
df_base_sp.sort_values(by = ['@id_DOG', 'DogName', 'Event_Dt'], inplace = True)


In [26]:

print(dognameslist[5])


BIG TOMMY NITRO


In [27]:

df_base_sp[df_base_sp['@id_DOG'].isin([108390981])]


,@id_DOG,Place,DogName,Box,Rug,Weight,StartPrice,Handicap,Margin1,Margin2,PIR,Checks,Comments,SplitMargin,RunTime,Prizemoney,RaceId,TrainerId,TrainerName,@id_RACE,RaceNum,RaceName,RaceTime,Distance,RaceGrade,Track,date,Event_Dt,ft_sp,x_num_dogs,position
3729,108390981,4,DOLLY DOES IT,5.0,5,30.4,$11.00,NaN,6.00,0.14,0,0,B3,NaN,26.43,80.0,626271182,92892,M Elphinstone,626271182,10,GREAT NORTHERN,03:14PM,452m,Mixed 4/5,Devonport,05 Jan 21,2021-01-05,11.0,8,4
9211,108390981,5,DOLLY DOES IT,4.0,4,30.2,$7.00,NaN,6.00,1.14,0,0,NaN,NaN,26.40,0.0,629797242,92892,M Elphinstone,629797242,10,GREAT NORTHERN,03:09PM,452m,Grade 5,Devonport,12 Jan 21,2021-01-12,7.0,7,5
14917,108390981,1,DOLLY DOES IT,5.0,5,30.2,$7.00,NaN,5.25,NaN,0,0,NaN,8.1,26.20,1890.0,629797513,92892,M Elphinstone,629797513,4,SUREPICK,01:27PM,452m,Grade 5,Devonport,19 Jan 21,2021-01-19,7.0,7,1
37110,108390981,7,DOLLY DOES IT,1.0,1,30.0,$7.00,NaN,10.50,2.00,0,0,NaN,NaN,26.48,0.0,638810808,92892,M Elphinstone,638810808,8,LADBROKES,02:22PM,452m,Mixed 4/5 Heat,Devonport,16 Feb 21,2021-02-16,7.0,8,7
48421,108390981,8,DOLLY DOES IT,8.0,8,29.9,$91.00,NaN,18.50,3.57,0,0,SO,NaN,26.91,0.0,643554706,92892,M Elphinstone,643554706,8,NEW RECRUIT 2012,02:22PM,452m,Grade 4,Devonport,02 Mar 21,2021-03-02,91.0,8,8


In [28]:

# ADDITIONAL VARIABLES
doglevel = ['@id_DOG']

# Distance and Speed in each race
df_base_sp['x_dist'] = df_base_sp['Distance'].apply(lambda x : x.replace('m','')).astype(int)
df_base_sp['speed'] = (df_base_sp['x_dist'] / df_base_sp['RunTime'])*1000

# Flag for getting a total race count
df_base_sp['f_1'] = 1

# Total Races
df_base_sp['x_races_tot'] = df_base_sp.groupby(doglevel).cumcount()


In [29]:

# Average Speed until prev race
df_base_sp['speed_prev'] = df_base_sp.groupby(doglevel)['speed'].shift(1)

df_base_sp['speed_avg'] = df_base_sp.groupby(doglevel)['speed_prev'].rolling(window=99, min_periods = 1).mean().reset_index(0,drop=True)
df_base_sp['speed_min'] = df_base_sp.groupby(doglevel)['speed_prev'].rolling(window=99, min_periods = 1).min().reset_index(0,drop=True)
df_base_sp['speed_max'] = df_base_sp.groupby(doglevel)['speed_prev'].rolling(window=99, min_periods = 1).max().reset_index(0,drop=True)

df_base_sp['speed_range'] = df_base_sp['speed_max'] - df_base_sp['speed_min']

df_base_sp['speed_std'] = df_base_sp.groupby(doglevel)['speed_prev'].rolling(window=99, min_periods = 1).std().reset_index(0,drop=True)

print(datetime.now())
      

2022-11-27 10:14:34.417652


In [30]:

# ADDITIONAL VARIABLES
grouplevel = ['@id_DOG','x_dist_gp']

# GROUPED X_DIST 
df_base_sp['x_dist_gp'] = df_base_sp['x_dist'].apply(lambda x : 300 if x < 330 else 350 if x < 380 else 400 if x < 420 else 450 if x < 485 else 500 if x < 550 else 600 if x < 650 else 700 if x < 750 else -999)
df_base_sp['x_runtm_gp'] = ( df_base_sp['RunTime'] / df_base_sp['x_dist'] ) * df_base_sp['x_dist_gp']

df_base_sp['x_rtgp_prev'] = df_base_sp.groupby(grouplevel)['x_runtm_gp'].shift(1)

#df_base_sp['x_rtgp_avg'] = df_base_sp.groupby(grouplevel)['x_rtgp_prev'].rolling(window=99, min_periods = 1).mean().reset_index(drop=True)
df_base_sp['x_rtgp_avg'] = df_base_sp.groupby(grouplevel)['x_rtgp_prev'].rolling(window=99, min_periods = 1).mean().values

#df_base_sp['x_rtgp_min'] = df_base_sp.groupby(grouplevel)['x_rtgp_prev'].rolling(window=99, min_periods = 1).min().reset_index(drop=True)
df_base_sp['x_rtgp_min'] = df_base_sp.groupby(grouplevel)['x_rtgp_prev'].rolling(window=99, min_periods = 1).min().values

#df_base_sp['x_rtgp_max'] = df_base_sp.groupby(grouplevel)['x_rtgp_prev'].rolling(window=99, min_periods = 1).max().reset_index(0,drop=True)
df_base_sp['x_rtgp_max'] = df_base_sp.groupby(grouplevel)['x_rtgp_prev'].rolling(window=99, min_periods = 1).max().values

df_base_sp['x_rtgp_range'] = df_base_sp['x_rtgp_max'] - df_base_sp['x_rtgp_min']

#df_base_sp['x_rtgp_std'] = df_base_sp.groupby(grouplevel)['x_rtgp_prev'].rolling(window=99, min_periods = 1).std().reset_index(0,drop=True)
df_base_sp['x_rtgp_std'] = df_base_sp.groupby(grouplevel)['x_rtgp_prev'].rolling(window=99, min_periods = 1).std().values

print(datetime.now())


2022-11-27 10:15:09.663727


In [31]:

#df_base_sp.head(25)

#df_base_sp[ df_base_sp['@id_DOG'].isin([108390981,108391383,108391431]) ].head(25)

df_base_sp[ df_base_sp['x_runtm_gp'].isna()].head()



,@id_DOG,Place,DogName,Box,Rug,Weight,StartPrice,Handicap,Margin1,Margin2,PIR,Checks,Comments,SplitMargin,RunTime,Prizemoney,RaceId,TrainerId,TrainerName,@id_RACE,RaceNum,RaceName,RaceTime,Distance,RaceGrade,Track,date,Event_Dt,ft_sp,x_num_dogs,position,x_dist,speed,f_1,x_races_tot,speed_prev,speed_avg,speed_min,speed_max,speed_range,speed_std,x_dist_gp,x_runtm_gp,x_rtgp_prev,x_rtgp_avg,x_rtgp_min,x_rtgp_max,x_rtgp_range,x_rtgp_std


In [32]:

print(df_base_sp.columns.values.tolist())


['@id_DOG', 'Place', 'DogName', 'Box', 'Rug', 'Weight', 'StartPrice', 'Handicap', 'Margin1', 'Margin2', 'PIR', 'Checks', 'Comments', 'SplitMargin', 'RunTime', 'Prizemoney', 'RaceId', 'TrainerId', 'TrainerName', '@id_RACE', 'RaceNum', 'RaceName', 'RaceTime', 'Distance', 'RaceGrade', 'Track', 'date', 'Event_Dt', 'ft_sp', 'x_num_dogs', 'position', 'x_dist', 'speed', 'f_1', 'x_races_tot', 'speed_prev', 'speed_avg', 'speed_min', 'speed_max', 'speed_range', 'speed_std', 'x_dist_gp', 'x_runtm_gp', 'x_rtgp_prev', 'x_rtgp_avg', 'x_rtgp_min', 'x_rtgp_max', 'x_rtgp_range', 'x_rtgp_std']


In [33]:

#temp = df_base_sp.groupby('x_dist').agg( {'@id_RACE':'nunique'} ).reset_index()
#temp.to_csv(f'{fileloc}/race_distance.csv', index = False)
#temp.sort_values(by = 'index').head()


In [34]:

print( len(dognameslist) )

print( len(df_base_sp['@id_DOG'].unique().tolist()) )

print(df_base_sp['speed'].isna().sum())
print(df_base_sp['speed_prev'].isna().sum())


25310
25295
0
25295


In [35]:

print(pd.DataFrame(df_base_sp.dtypes).transpose())


  @id_DOG   Place DogName      Box    Rug   Weight StartPrice Handicap  Margin1  Margin2     PIR Checks Comments SplitMargin  RunTime Prizemoney RaceId TrainerId TrainerName @id_RACE RaceNum  \
0   int64  object  object  float64  int64  float64     object  float64  float64  float64  object  int64   object     float64  float64    float64  int64     int64      object    int64   int64   

  RaceName RaceTime Distance RaceGrade   Track    date Event_Dt    ft_sp x_num_dogs position x_dist    speed    f_1 x_races_tot speed_prev speed_avg speed_min speed_max speed_range speed_std  \
0   object   object   object    object  object  object   object  float64      int64   object  int64  float64  int64       int64    float64   float64   float64   float64     float64   float64   

  x_dist_gp x_runtm_gp x_rtgp_prev x_rtgp_avg x_rtgp_min x_rtgp_max x_rtgp_range x_rtgp_std  
0     int64    float64     float64    float64    float64    float64      float64    float64  


In [36]:

print(pd.DataFrame(df_base_sp.isna().sum()).transpose())


   @id_DOG  Place  DogName  Box  Rug  Weight  StartPrice  Handicap  Margin1  Margin2    PIR  Checks  Comments  SplitMargin  RunTime  Prizemoney  RaceId  TrainerId  TrainerName  @id_RACE  RaceNum  \
0        0      0        0    0    0       1           0    536782        0    74620  71107       0    259643        90362        0      167442       0          0            0         0        0   

   RaceName  RaceTime  Distance  RaceGrade  Track  date  Event_Dt  ft_sp  x_num_dogs  position  x_dist  speed  f_1  x_races_tot  speed_prev  speed_avg  speed_min  speed_max  speed_range  speed_std  \
0        15         0         0          0      0     0         0      0           0         0       0      0    0            0       25295      25295      25295      25295        25295      49202   

   x_dist_gp  x_runtm_gp  x_rtgp_prev  x_rtgp_avg  x_rtgp_min  x_rtgp_max  x_rtgp_range  x_rtgp_std  
0          0           0        72071       72071       72071       72071         72071      129353 


### C) Treat Missing Values


In [37]:

# All Columns Realted to Average Speed

df_base_sp['x_no_prev_races'] = df_base_sp['speed_prev'].apply( lambda x : 1 if pd.isna(x) else 0)

print( df_base_sp['x_no_prev_races'].value_counts() )

df_base_sp['x_speed_prev'] = df_base_sp['speed_prev'].fillna(0)
df_base_sp['x_speed_avg'] = df_base_sp['speed_avg'].fillna(0)
df_base_sp['x_speed_min'] = df_base_sp['speed_min'].fillna(0)
df_base_sp['x_speed_max'] = df_base_sp['speed_max'].fillna(0)

df_base_sp['x_speed_range'] = df_base_sp['speed_range'].fillna(-1)

df_base_sp['x_speed_std'] = df_base_sp['speed_std'].fillna(0)
df_base_sp['x_speed_std'] = df_base_sp.apply(lambda x : -1 if x.x_no_prev_races == 1 else x.x_speed_std, axis = 1  )

print(datetime.now())


0    511487
1     25295
Name: x_no_prev_races, dtype: int64
2022-11-27 10:16:45.049660


In [38]:

# All Columns Related to RunTime in Group

df_base_sp['x_no_prev_r_in_dist'] = df_base_sp['x_rtgp_prev'].apply( lambda x : 1 if pd.isna(x) else 0)

print( df_base_sp['x_no_prev_r_in_dist'].value_counts() )

print(datetime.now())


0    464711
1     72071
Name: x_no_prev_r_in_dist, dtype: int64
2022-11-27 10:16:47.627467


In [39]:

rtgp_mvt = df_base_sp.groupby('@id_RACE').agg({ 'x_rtgp_prev':'mean', 'x_rtgp_avg':'mean', 'x_rtgp_min':'mean', 'x_rtgp_max':'mean', 'x_rtgp_range':'mean', 'x_rtgp_std':'mean'})\
            .reset_index()\
            .rename(columns = {'x_rtgp_prev':'mvt_rtgp_prev', 'x_rtgp_avg':'mvt_rtgp_avg', 'x_rtgp_min':'mvt_rtgp_min', 'x_rtgp_max':'mvt_rtgp_max', 'x_rtgp_range':'mvt_rtgp_range', 'x_rtgp_std':'mvt_rtgp_std'})

print(pd.DataFrame(rtgp_mvt.isna().sum()).transpose())


   @id_RACE  mvt_rtgp_prev  mvt_rtgp_avg  mvt_rtgp_min  mvt_rtgp_max  mvt_rtgp_range  mvt_rtgp_std
0         0            825           581           581           581             581          1608


In [41]:

print(pd.DataFrame(df_base_sp.isna().sum()).transpose())
print()
print()
print()

print(df_base_sp.shape)
df.ft_mvt = df_base_sp.merge(rtgp_mvt, on = '@id_RACE', how = 'left')
print(df.ft_mvt.shape)

df.ft_mvt['x_rtgp_prev'].fillna(df.ft_mvt['mvt_rtgp_prev'], inplace=True)

df.ft_mvt['x_rtgp_avg'].fillna(df.ft_mvt['mvt_rtgp_avg'], inplace=True)
df.ft_mvt['x_rtgp_min'].fillna(df.ft_mvt['mvt_rtgp_min'], inplace=True)
df.ft_mvt['x_rtgp_max'].fillna(df.ft_mvt['mvt_rtgp_max'], inplace=True)

df.ft_mvt['x_rtgp_range'].fillna(df.ft_mvt['mvt_rtgp_range'], inplace=True)
df.ft_mvt['x_rtgp_std'].fillna(df.ft_mvt['mvt_rtgp_std'], inplace=True)

print()
print()
print()
print(pd.DataFrame(df.ft_mvt.isna().sum()).transpose())


# MIGHT NEED TO CHECK THIS AGAIN
df.ft_mvt['x_rtgp_prev'].fillna(0, inplace=True)

df.ft_mvt['x_rtgp_avg'].fillna(0, inplace=True)
df.ft_mvt['x_rtgp_min'].fillna(0, inplace=True)
df.ft_mvt['x_rtgp_max'].fillna(0, inplace=True)

df.ft_mvt['x_rtgp_range'].fillna(0, inplace=True)
df.ft_mvt['x_rtgp_std'].fillna(0, inplace=True)

print()
print()
print()
print(pd.DataFrame(df.ft_mvt.isna().sum()).transpose())


   @id_DOG  Place  DogName  Box  Rug  Weight  StartPrice  Handicap  Margin1  Margin2    PIR  Checks  Comments  SplitMargin  RunTime  Prizemoney  RaceId  TrainerId  TrainerName  @id_RACE  RaceNum  \
0        0      0        0    0    0       1           0    536782        0    74620  71107       0    259643        90362        0      167442       0          0            0         0        0   

   RaceName  RaceTime  Distance  RaceGrade  ...  speed  f_1  x_races_tot  speed_prev  speed_avg  speed_min  speed_max  speed_range  speed_std  x_dist_gp  x_runtm_gp  x_rtgp_prev  x_rtgp_avg  \
0        15         0         0          0  ...      0    0            0       25295      25295      25295      25295        25295      49202          0           0        72071       72071   

   x_rtgp_min  x_rtgp_max  x_rtgp_range  x_rtgp_std  x_no_prev_races  x_speed_prev  x_speed_avg  x_speed_min  x_speed_max  x_speed_range  x_speed_std  x_no_prev_r_in_dist  
0       72071       72071         72071    

In [42]:

#df.ft_mvt[ df.ft_mvt.mvt_rtgp_prev.isna() | df.ft_mvt.x_rtgp_prev.isna()  ].head(25)


df.ft_mvt[ df.ft_mvt['@id_RACE'].isin([626271182, 626218628, 626240593])].sort_values(by = ['@id_RACE','Place']).head(25)


,@id_DOG,Place,DogName,Box,Rug,Weight,StartPrice,Handicap,Margin1,Margin2,PIR,Checks,Comments,SplitMargin,RunTime,Prizemoney,RaceId,TrainerId,TrainerName,@id_RACE,RaceNum,RaceName,RaceTime,Distance,RaceGrade,...,speed_max,speed_range,speed_std,x_dist_gp,x_runtm_gp,x_rtgp_prev,x_rtgp_avg,x_rtgp_min,x_rtgp_max,x_rtgp_range,x_rtgp_std,x_no_prev_races,x_speed_prev,x_speed_avg,x_speed_min,x_speed_max,x_speed_range,x_speed_std,x_no_prev_r_in_dist,mvt_rtgp_prev,mvt_rtgp_avg,mvt_rtgp_min,mvt_rtgp_max,mvt_rtgp_range,mvt_rtgp_std
16587,251076212,1,BURN OUT BOY,5.0,5,33.2,$2.60F,NaN,1.00,NaN,211,0,M21c1 Sw,6.62,22.23,1100.0,626218628,68894,G Price,626218628,3,Try The Chef's Special,06:52PM,380m,Grade 6,...,NaN,NaN,NaN,400,23.400000,0.0,0.0,0.0,0.0,0.0,0.0,1,0.0,0.0,0.0,0.0,-1.0,-1.0,1,NaN,NaN,NaN,NaN,NaN,NaN
175034,415996356,2,SCARY FUN,3.0,3,27.2,$4.00,NaN,1.00,1.14,332,0,Mc3c3c2,6.77,22.31,330.0,626218628,112106,L Isaac,626218628,3,Try The Chef's Special,06:52PM,380m,Grade 6,...,NaN,NaN,NaN,400,23.484211,0.0,0.0,0.0,0.0,0.0,0.0,1,0.0,0.0,0.0,0.0,-1.0,-1.0,1,NaN,NaN,NaN,NaN,NaN,NaN
31246,295446742,3,METRE EATER,2.0,2,33.1,$9.90,NaN,4.25,3.00,744,0,Sc7c44,6.88,22.52,165.0,626218628,72510,B McPherson,626218628,3,Try The Chef's Special,06:52PM,380m,Grade 6,...,NaN,NaN,NaN,400,23.705263,0.0,0.0,0.0,0.0,0.0,0.0,1,0.0,0.0,0.0,0.0,-1.0,-1.0,1,NaN,NaN,NaN,NaN,NaN,NaN
7123,219044234,4,REPEAT SET,8.0,8,30.4,$12.00,NaN,5.50,1.43,656,0,M65w6,6.84,22.62,55.0,626218628,127397,C Bainbridge,626218628,3,Try The Chef's Special,06:52PM,380m,Grade 6,...,NaN,NaN,NaN,400,23.810526,0.0,0.0,0.0,0.0,0.0,0.0,1,0.0,0.0,0.0,0.0,-1.0,-1.0,1,NaN,NaN,NaN,NaN,NaN,NaN
14587,246692403,5,KABARANGI,1.0,1,24.1,$40.60,NaN,5.75,0.29,565,0,Q5c65,6.84,22.64,0.0,626218628,291086,T Mitchem,626218628,3,Try The Chef's Special,06:52PM,380m,Grade 6,...,NaN,NaN,NaN,400,23.831579,0.0,0.0,0.0,0.0,0.0,0.0,1,0.0,0.0,0.0,0.0,-1.0,-1.0,1,NaN,NaN,NaN,NaN,NaN,NaN
5,108391383,6,SIMPLY GIFTED,6.0,6,24.8,$46.50,NaN,7.25,1.43,877,0,Sc8c77,6.93,22.74,0.0,626218628,291086,T Mitchem,626218628,3,Try The Chef's Special,06:52PM,380m,Grade 6,...,NaN,NaN,NaN,400,23.936842,0.0,0.0,0.0,0.0,0.0,0.0,1,0.0,0.0,0.0,0.0,-1.0,-1.0,1,NaN,NaN,NaN,NaN,NaN,NaN
10775,231587263,7,ABBEY SIOBHAN,4.0,4,30.8,$5.60,NaN,9.00,1.71,123,0,Q12cc3,6.59,22.86,0.0,626218628,102996,M Stewart,626218628,3,Try The Chef's Special,06:52PM,380m,Grade 6,...,NaN,NaN,NaN,400,24.063158,0.0,0.0,0.0,0.0,0.0,0.0,1,0.0,0.0,0.0,0.0,-1.0,-1.0,1,NaN,NaN,NaN,NaN,NaN,NaN
27818,289026900,8,CROWN BENJI,7.0,9,32.4,$6.80,NaN,18.25,9.29,488,0,M48to8 I-21,6.82,23.51,0.0,626218628,83951,R Noden,626218628,3,Try The Chef's Special,06:52PM,380m,Grade 6,...,NaN,NaN,NaN,400,24.747368,0.0,0.0,0.0,0.0,0.0,0.0,1,0.0,0.0,0.0,0.0,-1.0,-1.0,1,NaN,NaN,NaN,NaN,NaN,NaN
214258,434800047,1,IVY CONNIVING,8.0,8,28.2,$2.20F,NaN,8.75,NaN,111,0,111,8.20,22.55,1600.0,626240593,108360,S Sultana,626240593,6,STEELINE RIVERSTONE,04:52PM,400m,Mixed 4/5,...,NaN,NaN,NaN,400,22.550000,0.0,0.0,0.0,0.0,0.0,0.0,1,0.0,0.0,0.0,0.0,-1.0,-1.0,1,NaN,NaN,NaN,NaN,NaN,NaN
16944,253088672,2,SHIMMY SPARKLES,3.0,3,27.0,$19.00,NaN,8.75,8.71,544,0,544,8.50,23.16,500.0,626240593,114260,L Azzopardi,626240593,6,STEELINE RIVERSTONE,04:52PM,400m,Mixed 4/5,...,NaN,NaN,NaN,400,23.160000,0.0,0.0,0.0,0.0,0.0,0.0,1,0.0,0.0,0.0,0.0,-1.0,-1.0,1,NaN,NaN,NaN,NaN,NaN,NaN


In [43]:

print(df.ft_mvt.describe())


            @id_DOG            Box            Rug         Weight  Handicap        Margin1        Margin2         Checks    SplitMargin        RunTime      Prizemoney        RaceId      TrainerId  \
count  5.367820e+05  536782.000000  536782.000000  536781.000000       0.0  536782.000000  462162.000000  536782.000000  446420.000000  536782.000000   369340.000000  5.367820e+05  536782.000000   
mean   4.526384e+08       4.477762       4.584563      29.918423       NaN       6.449292       2.060021       0.576318       7.297971      24.401001      425.664710  7.292602e+08  109616.105311   
std    1.120535e+08       2.344809       2.432678       3.322921       NaN       8.979399       8.162029       1.647500       2.514391       4.949724     3016.878533  5.833205e+07   76306.821982   
min    1.083910e+08       1.000000       1.000000      18.900000       NaN       0.000000       0.000000       0.000000       0.000000      10.000000        0.000000  6.238619e+08    6418.000000   
25%    3.8

In [44]:

print( df.ft_mvt.columns.values.tolist() )


['@id_DOG', 'Place', 'DogName', 'Box', 'Rug', 'Weight', 'StartPrice', 'Handicap', 'Margin1', 'Margin2', 'PIR', 'Checks', 'Comments', 'SplitMargin', 'RunTime', 'Prizemoney', 'RaceId', 'TrainerId', 'TrainerName', '@id_RACE', 'RaceNum', 'RaceName', 'RaceTime', 'Distance', 'RaceGrade', 'Track', 'date', 'Event_Dt', 'ft_sp', 'x_num_dogs', 'position', 'x_dist', 'speed', 'f_1', 'x_races_tot', 'speed_prev', 'speed_avg', 'speed_min', 'speed_max', 'speed_range', 'speed_std', 'x_dist_gp', 'x_runtm_gp', 'x_rtgp_prev', 'x_rtgp_avg', 'x_rtgp_min', 'x_rtgp_max', 'x_rtgp_range', 'x_rtgp_std', 'x_no_prev_races', 'x_speed_prev', 'x_speed_avg', 'x_speed_min', 'x_speed_max', 'x_speed_range', 'x_speed_std', 'x_no_prev_r_in_dist', 'mvt_rtgp_prev', 'mvt_rtgp_avg', 'mvt_rtgp_min', 'mvt_rtgp_max', 'mvt_rtgp_range', 'mvt_rtgp_std']


In [45]:

print(datetime.now().date())


2022-11-27


In [46]:

# 'TrainerName', 'Track' have been dummified
cols_for_drop = ['Box','Rug','Weight','Handicap','Margin1','Margin2','PIR','Checks','Comments','SplitMargin','Prizemoney','RaceId','TrainerId','RaceTime','Distance','RaceGrade','date', 'f_1','speed',\
                'Place','StartPrice'\
                ,'speed_prev', 'speed_avg', 'speed_min', 'speed_max', 'speed_range', 'speed_std'\
                , 'DogName', 'TrainerName', 'RaceNum', 'RaceName', 'Track', 'Event_Dt', 'position','RunTime'\
                , 'x_dist','ft_sp', 'x_dist'\
                , 'mvt_rtgp_prev', 'mvt_rtgp_avg', 'mvt_rtgp_min' ,'mvt_rtgp_max' ,'mvt_rtgp_range', 'mvt_rtgp_std']

## SHOULD BE THE LAST STEP OR a Separate Code
#df.ft1 = df.ft.copy()
exp_ds = df.ft_mvt.drop(columns = cols_for_drop)

# removed the Categorical columns from the original datast, adds the dummy variables, creates a new dataset
print(exp_ds.columns)


Index(['@id_DOG', '@id_RACE', 'x_num_dogs', 'x_races_tot', 'x_dist_gp', 'x_runtm_gp', 'x_rtgp_prev', 'x_rtgp_avg', 'x_rtgp_min', 'x_rtgp_max', 'x_rtgp_range', 'x_rtgp_std', 'x_no_prev_races',
       'x_speed_prev', 'x_speed_avg', 'x_speed_min', 'x_speed_max', 'x_speed_range', 'x_speed_std', 'x_no_prev_r_in_dist'],
      dtype='object')


In [47]:

exp_ds.head()


,@id_DOG,@id_RACE,x_num_dogs,x_races_tot,x_dist_gp,x_runtm_gp,x_rtgp_prev,x_rtgp_avg,x_rtgp_min,x_rtgp_max,x_rtgp_range,x_rtgp_std,x_no_prev_races,x_speed_prev,x_speed_avg,x_speed_min,x_speed_max,x_speed_range,x_speed_std,x_no_prev_r_in_dist
0,108390981,626271182,8,0,450,26.313053,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1,0.000000,0.000000,0.000000,0.000000,-1.000000,-1.000000,1
1,108390981,629797242,7,1,450,26.283186,26.313053,26.313053,26.313053,26.313053,0.000000,0.000000,0,17101.778282,17101.778282,17101.778282,17101.778282,0.000000,0.000000,0
2,108390981,629797513,7,2,450,26.084071,26.283186,26.298119,26.283186,26.313053,0.029867,0.021119,0,17121.212121,17111.495202,17101.778282,17121.212121,19.433839,13.741799,0
3,108390981,638810808,8,3,450,26.362832,26.084071,26.226770,26.084071,26.313053,0.228982,0.124480,0,17251.908397,17158.299600,17101.778282,17251.908397,150.130115,81.647864,0
4,108390981,643554706,8,4,450,26.790929,26.362832,26.260785,26.084071,26.362832,0.278761,0.122305,0,17069.486405,17136.096301,17069.486405,17251.908397,182.421992,80.101155,0


### EXPORTING THE SPEED PROFILE csv

In [48]:

#keep_cols_for_export = ['@id_DOG', 'DogName', '@id_RACE','RaceName','Track','RaceGrade','Distance','TrainerName','RunTime','Place','x_races_tot']
#'Event_Dt', 

cols_for_drop = []

todaydt = datetime.now().date()
exp_fname = f'dataprep_speed_profile_{todaydt}.csv'


In [49]:

print(exp_ds.columns.values.tolist())


['@id_DOG', '@id_RACE', 'x_num_dogs', 'x_races_tot', 'x_dist_gp', 'x_runtm_gp', 'x_rtgp_prev', 'x_rtgp_avg', 'x_rtgp_min', 'x_rtgp_max', 'x_rtgp_range', 'x_rtgp_std', 'x_no_prev_races', 'x_speed_prev', 'x_speed_avg', 'x_speed_min', 'x_speed_max', 'x_speed_range', 'x_speed_std', 'x_no_prev_r_in_dist']


In [50]:

exp_ds.head()


,@id_DOG,@id_RACE,x_num_dogs,x_races_tot,x_dist_gp,x_runtm_gp,x_rtgp_prev,x_rtgp_avg,x_rtgp_min,x_rtgp_max,x_rtgp_range,x_rtgp_std,x_no_prev_races,x_speed_prev,x_speed_avg,x_speed_min,x_speed_max,x_speed_range,x_speed_std,x_no_prev_r_in_dist
0,108390981,626271182,8,0,450,26.313053,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1,0.000000,0.000000,0.000000,0.000000,-1.000000,-1.000000,1
1,108390981,629797242,7,1,450,26.283186,26.313053,26.313053,26.313053,26.313053,0.000000,0.000000,0,17101.778282,17101.778282,17101.778282,17101.778282,0.000000,0.000000,0
2,108390981,629797513,7,2,450,26.084071,26.283186,26.298119,26.283186,26.313053,0.029867,0.021119,0,17121.212121,17111.495202,17101.778282,17121.212121,19.433839,13.741799,0
3,108390981,638810808,8,3,450,26.362832,26.084071,26.226770,26.084071,26.313053,0.228982,0.124480,0,17251.908397,17158.299600,17101.778282,17251.908397,150.130115,81.647864,0
4,108390981,643554706,8,4,450,26.790929,26.362832,26.260785,26.084071,26.362832,0.278761,0.122305,0,17069.486405,17136.096301,17069.486405,17251.908397,182.421992,80.101155,0


In [51]:

exp_ds.describe()


,@id_DOG,@id_RACE,x_num_dogs,x_races_tot,x_dist_gp,x_runtm_gp,x_rtgp_prev,x_rtgp_avg,x_rtgp_min,x_rtgp_max,x_rtgp_range,x_rtgp_std,x_no_prev_races,x_speed_prev,x_speed_avg,x_speed_min,x_speed_max,x_speed_range,x_speed_std,x_no_prev_r_in_dist
count,5.367820e+05,5.367820e+05,536782.000000,536782.000000,536782.000000,536782.000000,536782.000000,536782.000000,536782.000000,536782.000000,536782.000000,536782.000000,536782.000000,536782.000000,536782.000000,536782.000000,536782.000000,536782.000000,536782.000000,536782.000000
mean,4.526384e+08,7.292602e+08,7.336054,17.441984,415.650767,24.331163,24.057037,24.026348,23.691691,24.470661,0.778971,0.307534,0.047123,16297.732893,16295.338504,15814.290620,16692.555000,878.217257,255.179679,0.134265
std,1.120535e+08,5.833205e+07,0.896384,15.125087,77.953580,4.640764,5.226176,4.902033,4.862822,4.974698,0.694263,0.200529,0.211903,3648.298680,3638.174331,3549.703372,3754.481068,694.694870,202.407246,0.340937
min,1.083910e+08,6.238619e+08,5.000000,0.000000,300.000000,9.950249,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-1.000000,-1.000000,0.000000
25%,3.836043e+08,6.801957e+08,7.000000,5.000000,350.000000,20.735650,20.619335,20.664857,20.404899,21.095455,0.351852,0.212253,0.000000,16801.760096,16879.770772,16326.530612,17226.148410,539.636695,183.157045,0.000000
50%,4.517679e+08,7.290297e+08,8.000000,14.000000,400.000000,23.700000,23.659794,23.592500,23.250302,24.111392,0.729114,0.286712,0.000000,17088.326676,17104.449778,16618.263956,17508.545385,853.719448,246.661374,0.000000
75%,5.274578e+08,7.792436e+08,8.000000,26.000000,450.000000,27.167053,27.031323,26.730624,26.413043,27.261021,1.086538,0.371974,0.000000,17350.395183,17291.111837,16857.610475,17740.981668,1144.892476,315.507470,0.000000
max,2.145360e+09,8.292517e+08,8.000000,117.000000,700.000000,384.582524,61.747368,46.248951,46.248951,61.747368,39.167368,10.132302,1.000000,40200.000000,40200.000000,40200.000000,40200.000000,23826.557771,16614.431948,1.000000


In [52]:

exp_ds.to_csv(f'{fileloc}/{exp_fname}', index = False)


In [53]:

print('THE END')


THE END
